In [1]:
%pip install -q -U fla-core


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install transformers tokenizers accelerate huggingface_hub safetensors


Note: you may need to restart the kernel to use updated packages.


In [3]:
# %pip install --no-cache-dir --force-reinstall \
#   "transformers==4.46.3" "tokenizers>=0.20.0" "accelerate>=0.33.0" \
#   "huggingface_hub>=0.24.0" "safetensors"


In [4]:
%pip install -q --no-cache-dir --force-reinstall "numpy<2"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapters 1.1.0 requires transformers~=4.47.1, but you have transformers 4.57.3 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

messages = [
    {"role": "system", "content": "Du bist ein präziser Assistent für deutsche Textbearbeitung."},
    {"role": "user", "content": "Bitte fasse den folgenden Text in 5 Stichpunkten zusammen: ..."},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

out = model.generate(**inputs, max_new_tokens=25600, temperature=0.2)
print(tokenizer.decode(out[0], skip_special_tokens=True))


`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


system
Du bist ein präziser Assistent für deutsche Textbearbeitung.
user
Bitte fasse den folgenden Text in 5 Stichpunkten zusammen: ...
assistant
Ich kann nicht verstehen, was du dazu beziehst. Bitte erkläre, was du möchtest, um eine 5-stück-Zusammenfassung zu erzielen.


In [6]:
!pip -q install -U transformers accelerate bitsandbytes



In [7]:
!pip install -U "transformers>=4.57.1" accelerate huggingface_hub

  Using cached huggingface_hub-1.2.4-py3-none-any.whl.metadata (13 kB)


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Qwen/Qwen2.5-7B-Instruct"  # oder: "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
)

def get_cleaned_txt(txt):

    messages = [
        {"role": "system", "content": "You are working on a transcription of a video. You are only transcribing one speaker and not a complete dialog. Attached you find a first draft of the transcription. The draft probably includes mistakes due to rar or domain specific words. You should do two things. First, review the attached transcription and summarise what the transcription is about. Second, based on your own summary, rewrite the transcription and exchange words which might be wrong due to the context of the overall transcription. Only exchange certain words with better fitting words which may sound similar from a pronounciation perspective. Keep the structure of the attached file as it is. Keep the timestamps as they are. It is important that the transcription is complete. Do not leave any parts out. Do not hallucinate."},
        {"role": "user", "content": f"{txt}"},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    out = model.generate(**inputs, max_new_tokens=20000, temperature=0.2)
    return (tokenizer.decode(out[0], skip_special_tokens=True))

get_cleaned_txt("Formuliere diesen Text professioneller um (max. 120 Wörter): ...")



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

'system\nYou are working on a transcription of a video. You are only transcribing one speaker and not a complete dialog. Attached you find a first draft of the transcription. The draft probably includes mistakes due to rar or domain specific words. You should do two things. First, review the attached transcription and summarise what the transcription is about. Second, based on your own summary, rewrite the transcription and exchange words which might be wrong due to the context of the overall transcription. Only exchange certain words with better fitting words which may sound similar from a pronounciation perspective. Keep the structure of the attached file as it is. Keep the timestamps as they are. It is important that the transcription is complete. Do not leave any parts out. Do not hallucinate.\nuser\nFormuliere diesen Text professioneller um (max. 120 Wörter): ...\nassistant\nIch verstehe, dass Sie eine professionellere Fassung eines Textes in etwa 120 Wörtern haben möchten. Bitte 